# dcm mr -> bmp mr
# dcm ct -> bmp ct

# bmp mr + bmp ct -> paired img 로 만들어줌

In [17]:
# ========================
# ct, mr 리스트 가져오기
# ========================
rootpath='../rawdata/20210705_added_15/dcm/'
path_ct_li=[]
path_mr_li=[]
for path, dirs, files in os.walk(rootpath):
    if path.split('/')[-1]=='ct':
        path_ct_li.append(path)
    elif path.split('/')[-1]=='mr':
        path_mr_li.append(path)

path_li=[]
for i in range(len(path_ct_li)):
    path_li.append(path_ct_li[i][:-2])

In [330]:
## 한개폴더 기준으로 함수 만들어 따로 포문을 돌리자

from PIL import Image
import pydicom as dicom
import numpy as np
import os,glob,cv2
import matplotlib.pyplot as plt

def get_LUT_value(data, window, level):
    """Apply the RGB Look-Up Table for the given
       data and window/level value."""
    try:
        window = window[0]
    except TypeError:
        pass
    try:
        level = level[0]
    except TypeError:
        pass

    return np.piecewise(data,
                        [data <= (level - 0.5 - (window - 1) / 2),
                         data > (level - 0.5 + (window - 1) / 2)],
                        [0, 255, lambda data: ((data - (level - 0.5)) /
                         (window - 1) + 0.5) * (255 - 0)])

def dcmToJpg(path_dir,jpg_path_dir):

    file_list=os.listdir(path_dir)
    file_list.sort()

    dcm_list=[]
    for item in file_list:
        if item.find('.dcm') is not -1:
            dcm_list.append(item)
            print(item)
            
    for j in range(len(dcm_list)):
        ds=dicom.read_file(path_dir+"%s"%dcm_list[j])
        img = ds.pixel_array
        img=img.astype('float32')
        img_1024=img
        #image=get_LUT_value(img_1024, ds.WindowWidth, ds.WindowCenter)
        image=get_LUT_value(img_1024, 1500, 450) ########### 여기 바꿈 임의로
        ds_jpg=Image.fromarray(image).convert('L')
        ds_jpg.save(jpg_path_dir+"%s"%dcm_list[j][:-4]+".bmp")
        print(str(j+1)+"/"+str(len(dcm_list)))
        
        
        
def dcm2bmp_tmp(path_dir, bmp_path, foldername, bool_save):
    
    # ====================
    # CT
    # ====================
    file_list1=glob.glob(path_dir+'ct/*')
    #file_list1.sort()
    
    for file in file_list1:
        if file.split('.')[-1]!='dcm':
            os.rename(file, file+'.dcm')
        
    dcm_list1=[]
    for item in file_list1:
        if item.find('.dcm') is not -1:
            dcm_list1.append(item)
   
    print('len(ct)=',len(dcm_list1))
    
    # ====================
    # MR
    # ====================
    file_list2=glob.glob(path_dir+'mr/*')
    #file_list2.sort()
    
    for file in file_list2:
        if file.split('.')[-1]!='dcm':
            os.rename(file, file+'.dcm')
        
    dcm_list2=[]
    for item in file_list2:
        if item.find('.dcm') is not -1:
            dcm_list2.append(item)
            
    
    print('len(mr)=',len(dcm_list2))
    
    
    
    idx1=[]
    bmp_ct=[]
    for j in range(len(dcm_list1)): ##############
        # ====================
        # CT
        # ====================
        
        ds=dicom.read_file(dcm_list1[j])
        img = ds.pixel_array
        img_output= ds.RescaleSlope * img + ds.RescaleIntercept
        
        img_output=img_output.astype('float32')
        img_1024=img_output
        #image1=get_LUT_value(img_1024, ds.WindowWidth, ds.WindowCenter)
        image1=get_LUT_value(img_1024, 1500, 450)
        #print(j,image1.shape)
        image1=cv2.resize(image1,(512,512))
        #print(j,image1.shape)
        bmp_ct.append(image1)
        idx1.append(float(ds.ImagePositionPatient[2]))
        
        del image1
    
    idx1 = np.array(idx1,dtype=float)
    bmp_ct = [x for _, x in sorted(zip(idx1, bmp_ct),key = lambda x : x[0])]
    
    
    
    

    idx2=[]
    bmp_mr=[]
    for j in range(len(dcm_list2)):    ##############
        # ====================
        # MR
        # ====================
        ds2=dicom.read_file(dcm_list2[j]) #########
        img2 = ds2.pixel_array
        img_output2=img2
        img_output2= ds2.RescaleSlope * img2 + ds2.RescaleIntercept
        img_output2=img_output2.astype('float32')
        img_1024_2=img_output2
        #image2=get_LUT_value(img_1024_2, ds2.WindowWidth, ds2.WindowCenter)
        image2=get_LUT_value(img_1024_2, 670,335)
        #print(j, image2.shape)
        image2=cv2.resize(image2,(512,512))
        #print(j,image2.shape)
        bmp_mr.append(image2)
        idx2.append(float(ds2.ImagePositionPatient[2]))
        
        del image2
    
    idx2 = np.array(idx2,dtype=float)
    bmp_mr = [x for _, x in sorted(zip(idx2, bmp_mr),key = lambda x : x[0])]
       
    
    min_len=min(len(dcm_list1),len(dcm_list2))
    
    for j in range(min_len):
        print(j)
        k=j
        plt.subplot(1,2,1)
        plt.imshow(bmp_ct[j],cmap='gray')
        plt.subplot(1,2,2)
        plt.imshow(bmp_mr[k],cmap='gray')
        plt.show()
        
        
        if bool_save==True:
        # j 확인하고 나서 저장할것!!
            cv2.imwrite(bmp_path+'ct/%s_{0:03d}.bmp'.format(j)%(foldername),bmp_ct[j])
            cv2.imwrite(bmp_path+'mr/%s_{0:03d}.bmp'.format(j)%(foldername),bmp_mr[k])
        


In [329]:
#한개폴더만 test

num=15 ########

path_dir=path_li[num]
print(path_dir)
foldername=path_dir.split('/')[4]
bmp_path="../rawdata/20210705_added_15/bmp/"

dcm2bmp_tmp(path_dir,bmp_path,foldername,False) ########

#36,190

IndexError: list index out of range

In [333]:
# ====================
# hconcat
# ====================

ct_path='../rawdata/20210705_added_15/bmp/ct/*.bmp'
mr_path='../rawdata/20210705_added_15/bmp/mr/*.bmp'
all_ctmr='../rawdata/20210705_added_15/bmp/all_ctmr/'
ct_li=glob.glob(ct_path)
mr_li=glob.glob(mr_path)

for i in range(len(ct_li)):
    img_ct=cv2.imread(ct_li[i],cv2.IMREAD_GRAYSCALE)
    img_mr=cv2.imread(mr_li[i],cv2.IMREAD_GRAYSCALE)
    img3=cv2.hconcat([img_ct,img_mr])
    filename=ct_li[i].split('/')[-1]
    cv2.imwrite(all_ctmr+filename,img3)
    print(all_ctmr+filename)
    del img_ct, img_mr, img3

../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_011.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_012.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_013.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_014.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_015.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_016.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_017.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_018.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_019.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_020.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_021.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_022.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_023.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_024.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_025.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/03_35810823_026.bmp
../rawda

../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_046.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_047.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_048.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_049.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_050.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_051.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_052.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_053.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_054.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_055.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_056.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_057.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_058.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_059.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_060.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_28976523_061.bmp
../rawda

../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_112.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_113.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_114.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_115.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_116.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_117.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_118.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_119.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_120.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_121.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_122.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_123.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_124.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_125.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_126.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/05_32630123_127.bmp
../rawda

../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_089.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_090.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_091.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_092.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_093.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_094.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_095.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_096.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_097.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_098.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_099.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_100.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_101.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_102.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_103.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_13862663_104.bmp
../rawda

../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_019.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_020.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_021.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_022.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_023.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_024.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_025.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_026.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_027.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_028.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_029.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_030.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_031.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_032.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_033.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/06_28977003_034.bmp
../rawda

...

../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_050.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_051.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_052.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_053.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_054.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_055.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_056.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_057.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_058.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_059.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_060.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_061.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_062.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_063.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_064.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_09364403_065.bmp
../rawda

../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_010.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_011.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_012.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_013.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_014.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_015.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_016.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_017.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_018.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_019.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_020.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_021.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_022.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_023.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-1_024.bmp
../rawdata/20210705_added_15/bmp/all_ctm

../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_051.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_052.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_053.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_054.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_055.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_056.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_057.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_058.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_059.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_060.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_061.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_062.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_063.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_064.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_35042243-2_065.bmp
../rawdata/20210705_added_15/bmp/all_ctm

../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_106.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_107.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_108.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_109.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_110.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_111.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_112.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_113.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_114.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_115.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_116.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_117.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_118.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_119.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_120.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40143013_121.bmp
../rawda

../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_065.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_066.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_067.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_068.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_069.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_070.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_071.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_072.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_073.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_074.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_075.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_076.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_077.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_078.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_079.bmp
../rawdata/20210705_added_15/bmp/all_ctmr/08_40363273_080.bmp
../rawda